In [1]:
%run dataFormating.ipynb

rmdfTestUsers read_csv success (1/3)
rmdf1522 read_csv success (2/3)
rmdf160 read_csv success (3/3)
gform read_csv success
gformFR read_csv success
temporalities set (user answer method)


IntProgress(value=0, max=1708)

IntProgress(value=0, max=854)

profile info set


FloatProgress(value=0.0, max=621.0)

dataFormating.ipynb:9: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  "outputs": [


# What subsets of scientific questions tend to be answered correctly by the same subjects?

## Mining

In [2]:
from orangecontrib.associate.fpgrowth import *  
import pandas as pd
from numpy import *

In [3]:
questions = correctedScientific.columns
correctedScientificText = [[] for _ in range(correctedScientific.shape[0])]
for q in questions:
    for index in range(correctedScientific.shape[0]):
        r = correctedScientific.index[index]
        if correctedScientific.loc[r, q]:
            correctedScientificText[index].append(q)
#correctedScientificText

In [4]:
len(correctedScientificText)

252

In [5]:
# Get frequent itemsets with support > 25%
# run time < 1 min
itemsets = frequent_itemsets(correctedScientificText, math.floor(len(correctedScientificText) * 0.25))
#dict(itemsets)

In [6]:
# Generate rules according to confidence, confidence > 85 %
# run time < 5 min
rules = association_rules(dict(itemsets), 0.85)
#list(rules)

In [7]:
# Transform rules generator into a Dataframe
rulesDataframe = pd.DataFrame([(ant, cons, supp, conf) for ant, cons, supp, conf in rules])
rulesDataframe.rename(columns = {0:"antecedants", 1:"consequents", 2:"support", 3:"confidence"}, inplace=True)
rulesDataframe.head()

,antecedants,consequents,support,confidence
0,(QDeviceRbsPconsAmprTer),(QDeviceRbsPconsFlhdcTer),65,0.984848
1,(QUnequipDevice),(QGenotypePhenotype),66,0.868421
2,(QBioBricksDevicesComposition),(QGenotypePhenotype),69,0.907895
3,(QDeviceRbsPconsFlhdcTer),(QGenotypePhenotype),71,0.865854


In [8]:
# Save the mined rules to file
rulesDataframe.to_csv("results/associationRulesMiningSupport25percentsConfidence85percents.csv")

## Search for interesting rules
Interesting rules are more likely to be the ones with highest confidence, the highest lift or with a bigger consequent set. Pairs can also be especially interesting

In [9]:
# Sort rules by confidence
confidenceSortedRules = rulesDataframe.sort_values(by = ["confidence", "support"], ascending=[False, False])
confidenceSortedRules.head(50)

,antecedants,consequents,support,confidence
0,(QDeviceRbsPconsAmprTer),(QDeviceRbsPconsFlhdcTer),65,0.984848
2,(QBioBricksDevicesComposition),(QGenotypePhenotype),69,0.907895
1,(QUnequipDevice),(QGenotypePhenotype),66,0.868421
3,(QDeviceRbsPconsFlhdcTer),(QGenotypePhenotype),71,0.865854


In [10]:
# Sort rules by size of consequent set
rulesDataframe["consequentSize"] = rulesDataframe["consequents"].apply(lambda x: len(x))
consequentSortedRules = rulesDataframe.sort_values(by = ["consequentSize", "confidence", "support"], ascending=[False, False, False])
consequentSortedRules.head(50)

,antecedants,consequents,support,confidence,consequentSize
0,(QDeviceRbsPconsAmprTer),(QDeviceRbsPconsFlhdcTer),65,0.984848,1
2,(QBioBricksDevicesComposition),(QGenotypePhenotype),69,0.907895,1
1,(QUnequipDevice),(QGenotypePhenotype),66,0.868421,1
3,(QDeviceRbsPconsFlhdcTer),(QGenotypePhenotype),71,0.865854,1


In [11]:
# Select only pairs (rules with antecedent and consequent of size one)
# Sort pairs according to confidence
rulesDataframe["fusedRule"] = rulesDataframe[["antecedants", "consequents"]].apply(lambda x: frozenset().union(*x), axis=1)
rulesDataframe["ruleSize"] = rulesDataframe["fusedRule"].apply(lambda x: len(x))
pairRules = rulesDataframe.sort_values(by=["ruleSize", "confidence", "support"], ascending=[True, False, False])
pairRules.head(30)

,antecedants,consequents,support,confidence,consequentSize,fusedRule,ruleSize
0,(QDeviceRbsPconsAmprTer),(QDeviceRbsPconsFlhdcTer),65,0.984848,1,"(QDeviceRbsPconsFlhdcTer, QDeviceRbsPconsAmprTer)",2
2,(QBioBricksDevicesComposition),(QGenotypePhenotype),69,0.907895,1,"(QGenotypePhenotype, QBioBricksDevicesComposit...",2
1,(QUnequipDevice),(QGenotypePhenotype),66,0.868421,1,"(QUnequipDevice, QGenotypePhenotype)",2
3,(QDeviceRbsPconsFlhdcTer),(QGenotypePhenotype),71,0.865854,1,"(QDeviceRbsPconsFlhdcTer, QGenotypePhenotype)",2


In [12]:
correctedScientific.columns

Index(['QGenotypePhenotype', 'QBioBricksDevicesComposition', 'QAmpicillin',
       'QBBNamePlasmid', 'QBBFunctionTER', 'QBBNamePromoter',
       'QBBFunctionGameCDS', 'QBBNameTerminator', 'QBBFunctionBiologyCDS',
       'QBBNameRBS', 'QBBExampleCDS', 'QBBNameCDS', 'QBBFunctionPR',
       'QBBFunctionRBS', 'QBBFunctionPLASMID', 'QBBNameOperator',
       'QDeviceRbsPconsFlhdcTer', 'QDevicePconsRbsFlhdcTer',
       'QDevicePbadRbsGfpTer', 'QDevicePbadGfpTbsTer', 'QDeviceGfpRbsPconsTer',
       'QDevicePconsGfpRbsTer', 'QDeviceAmprRbsPconsTer',
       'QDeviceRbsPconsAmprTer', 'QGreenFluorescence', 'QUnequipDevice',
       'QDevicePbadRbsAraTer'],
      dtype='object')

In [13]:
# Sort questions by number of apparition in consequents
for q in scientificQuestions:
    rulesDataframe[q+"c"] = rulesDataframe["consequents"].apply(lambda x: 1 if q in x else 0)
occurenceInConsequents = rulesDataframe.loc[:,scientificQuestions[0]+"c":scientificQuestions[-1]+"c"].sum(axis=0)

occurenceInConsequents.sort_values(inplace=True, ascending=False)
occurenceInConsequents

QGenotypePhenotypec              3
QDeviceRbsPconsFlhdcTerc         1
QBBFunctionPRc                   0
QBioBricksDevicesCompositionc    0
QAmpicillinc                     0
QBBNamePlasmidc                  0
QBBFunctionTERc                  0
QBBNamePromoterc                 0
QBBFunctionGameCDSc              0
QBBNameTerminatorc               0
QBBFunctionBiologyCDSc           0
QBBNameRBSc                      0
QBBExampleCDSc                   0
QBBNameCDSc                      0
QDevicePbadRbsAraTerc            0
QUnequipDevicec                  0
QBBFunctionPLASMIDc              0
QBBNameOperatorc                 0
QDevicePconsRbsFlhdcTerc         0
QDevicePbadRbsGfpTerc            0
QDevicePbadGfpTbsTerc            0
QDeviceGfpRbsPconsTerc           0
QDevicePconsGfpRbsTerc           0
QDeviceAmprRbsPconsTerc          0
QDeviceRbsPconsAmprTerc          0
QGreenFluorescencec              0
QBBFunctionRBSc                  0
dtype: int64

In [14]:
# Sort questions by number of apparition in antecedants
for q in scientificQuestions:
    rulesDataframe[q+"a"] = rulesDataframe["antecedants"].apply(lambda x: 1 if q in x else 0)
occurenceInAntecedants = rulesDataframe.loc[:,scientificQuestions[0]+"a":scientificQuestions[-1]+"a"].sum(axis=0)
occurenceInAntecedants.sort_values(inplace=True, ascending=False)
occurenceInAntecedants

QBioBricksDevicesCompositiona    1
QDeviceRbsPconsAmprTera          1
QDeviceRbsPconsFlhdcTera         1
QUnequipDevicea                  1
QDevicePbadRbsAraTera            0
QBBNameCDSa                      0
QAmpicillina                     0
QBBNamePlasmida                  0
QBBFunctionTERa                  0
QBBNamePromotera                 0
QBBFunctionGameCDSa              0
QBBNameTerminatora               0
QBBFunctionBiologyCDSa           0
QBBNameRBSa                      0
QBBExampleCDSa                   0
QBBFunctionRBSa                  0
QBBFunctionPRa                   0
QBBFunctionPLASMIDa              0
QBBNameOperatora                 0
QDevicePconsRbsFlhdcTera         0
QDevicePbadRbsGfpTera            0
QDevicePbadGfpTbsTera            0
QDeviceGfpRbsPconsTera           0
QDevicePconsGfpRbsTera           0
QDeviceAmprRbsPconsTera          0
QGreenFluorescencea              0
QGenotypePhenotypea              0
dtype: int64